In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

raw_data = pd.read_csv('./Datset/Feature En.csv')

print(raw_data.shape)
df = raw_data.copy()
raw_data.head()

(10000, 18)


,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,Risk_Score,Loyalty Score
0,15634602,0.538,France,Female,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1.0,1.0,0.25,DIAMOND,0.391600,49.729730,54.0
1,15647311,0.516,Spain,Female,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,1.0,0.50,DIAMOND,0.382520,52.664639,52.0
2,15619304,0.304,France,Female,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1.0,1.0,0.50,DIAMOND,0.292849,29.426635,36.0
3,15701354,0.698,France,Female,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,0.0,1.00,GOLD,0.262202,15.180180,12.0
4,15737888,1.000,Spain,Female,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,0.0,1.00,GOLD,0.347333,55.137597,54.0


## Scilling Columns

In [19]:
col_features_minmax = [ 'Risk_Score','Loyalty Score']

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[col_features_minmax] = scaler.fit_transform(df[col_features_minmax])

df_scaled.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,Risk_Score,Loyalty Score
0,15634602,0.538,France,Female,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1.0,1.0,0.25,DIAMOND,0.391600,0.599531,0.54
1,15647311,0.516,Spain,Female,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,1.0,0.50,DIAMOND,0.382520,0.635204,0.52
2,15619304,0.304,France,Female,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1.0,1.0,0.50,DIAMOND,0.292849,0.352749,0.36
3,15701354,0.698,France,Female,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,0.0,1.00,GOLD,0.262202,0.179586,0.12
4,15737888,1.000,Spain,Female,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,0.0,1.00,GOLD,0.347333,0.665263,0.54


## Model Linear Regression

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Features
X_columns = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard",
             "IsActiveMember", "EstimatedSalary", "Complain", "Satisfaction Score",
             "Point Earned", "Risk_Score", "Loyalty Score"]
# Targets 
y_columns = ["Exited", "Risk_Score", "Loyalty Score"]

# Train (80%) และ Test (20%)
df_train, df_test = train_test_split(df_scaled, test_size=0.2, random_state=42)

predictions = []
mae_results = []

# Train และ Predict สำหรับแต่ละ Target
for target in y_columns:
    X_train = df_train[X_columns]
    y_train = df_train[target]
    X_test = df_test[X_columns]
    y_test = df_test[target]

    # สร้างโมเดล Linear Regression
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # คำนวณค่า MAE
    mae = mean_absolute_error(y_test, y_pred)

    # เก็บค่า MAE
    mae_results.append({"Target": target, "MAE": mae})

    # เก็บค่าที่ทำนายได้
    for i in range(len(y_test)):
        predictions.append({
            "Index": df_test.index[i],  # บันทึก Index ของข้อมูล
            "Target": target,
            "Predicted": y_pred[i],
            "Actual": y_test.values[i]
        })

predictions_df = pd.DataFrame(predictions).to_csv("./Result/Prediction_linear_regression.csv", index=False)
mae_df = pd.DataFrame(mae_results).to_csv("./Result/MAE_linear_regression.csv", index=False)

print("Model Linear Regression Finish! CSV File")

Model Linear Regression Finish! CSV File


In [36]:
df_perd = pd.read_csv('./Result/Prediction_linear_regression.csv')
df_mae = pd.read_csv('./Result/MAE_linear_regression.csv')

In [38]:
df_mae

,Target,MAE
0,Exited,3.404788e-03
1,Risk_Score,2.722388e-16
2,Loyalty Score,2.252374e-16


In [42]:
df_perd

,Index,Target,Predicted,Actual
0,6252,Exited,0.000764,0.00
1,4684,Exited,0.000971,0.00
2,1731,Exited,0.004438,0.00
3,4742,Exited,0.000916,0.00
4,4521,Exited,-0.002319,0.00
...,...,...,...,...
5995,6412,Loyalty Score,0.600000,0.60
5996,8285,Loyalty Score,0.700000,0.70
5997,7853,Loyalty Score,0.140000,0.14
5998,1095,Loyalty Score,0.080000,0.08


## แยกแต่ละ Target เป็นไฟล์

In [53]:
for target in y_columns:
    df_target = df_perd[df_perd["Target"] == target] 
    file_name = f"./Result/linear_regression_predictions_{target}.csv"  
    df_target.to_csv(file_name, index=False)  

print("Finish! CSV File")

Finish! CSV File


In [55]:
df_exited = pd.read_csv('./Result/linear_regression_predictions_Exited.csv')
df_loyalty = pd.read_csv('./Result/linear_regression_predictions_Loyalty Score.csv')
df_risk_score = pd.read_csv('./Result/linear_regression_predictions_Risk_Score.csv')

In [57]:
df_exited

,Index,Target,Predicted,Actual
0,6252,Exited,0.000764,0.0
1,4684,Exited,0.000971,0.0
2,1731,Exited,0.004438,0.0
3,4742,Exited,0.000916,0.0
4,4521,Exited,-0.002319,0.0
...,...,...,...,...
1995,6412,Exited,0.993679,1.0
1996,8285,Exited,-0.001890,0.0
1997,7853,Exited,0.997470,1.0
1998,1095,Exited,0.993717,1.0


In [59]:
df_loyalty

,Index,Target,Predicted,Actual
0,6252,Loyalty Score,0.16,0.16
1,4684,Loyalty Score,0.62,0.62
2,1731,Loyalty Score,0.18,0.18
3,4742,Loyalty Score,0.76,0.76
4,4521,Loyalty Score,0.64,0.64
...,...,...,...,...
1995,6412,Loyalty Score,0.60,0.60
1996,8285,Loyalty Score,0.70,0.70
1997,7853,Loyalty Score,0.14,0.14
1998,1095,Loyalty Score,0.08,0.08


In [61]:
df_risk_score

,Index,Target,Predicted,Actual
0,6252,Risk_Score,0.191943,0.191943
1,4684,Risk_Score,0.685491,0.685491
2,1731,Risk_Score,0.204224,0.204224
3,4742,Risk_Score,0.822057,0.822057
4,4521,Risk_Score,0.586171,0.586171
...,...,...,...,...
1995,6412,Risk_Score,0.701342,0.701342
1996,8285,Risk_Score,0.515761,0.515761
1997,7853,Risk_Score,0.137974,0.137974
1998,1095,Risk_Score,0.049277,0.049277


## Accuracy Score R²

In [69]:
from sklearn.metrics import r2_score

for target in y_columns:
    df_target = df_perd[df_perd["Target"] == target]
    r2 = r2_score(df_target["Actual"], df_target["Predicted"])
    print(f"R² Score Accuracy {target} : {r2*100:.2f} %")

R² Score Accuracy Exited : 99.37 %
R² Score Accuracy Risk_Score : 100.00 %
R² Score Accuracy Loyalty Score : 100.00 %
